# GeoBench BigEarthNetV2

In [1]:
import matplotlib.pyplot as plt

mkdir -p failed for path /home/toolkit/.cache/matplotlib: [Errno 13] Permission denied: '/home/toolkit/.cache/matplotlib'
Matplotlib created a temporary cache directory at /tmp/matplotlib-tyncq8ln because there was an issue with the default path (/home/toolkit/.cache/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [6]:
from geobench_v2.datamodules import GeoBenchBENV2DataModule
import tacoreader

In [5]:
band_order = {"s2": ["B4", "B3", "B2", 1.5], "s1": ["VV", "VH", 0.7]}

datamodule = GeoBenchBENV2DataModule(
    img_size=224,
    batch_size=16,
    num_workers=4,
    root="/mnt/rg_climate_benchmark/data/geobenchV2/benv2",
    band_order=band_order,
)
datamodule.setup("fit")

Error processing file /mnt/rg_climate_benchmark/data/geobenchV2/benv2/FullBenV2.0002.part.tortilla: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.


ValueError: Could not resolve band B4 for modality s2
Available bands: Available bands:

s2:
  - B01 (coastal): coastal_aerosol, b01
  - B02 (blue): b02, blue
  - B03 (green): b03, green
  - B04 (red): b04, red
  - B05 (vegetation_red_edge_1): re1, b05
  - B06 (vegetation_red_edge_2): re2, b06
  - B07 (vegetation_red_edge_3): re3, b07
  - B08 (nir): near_infrared, b08
  - B8A (vegetation_red_edge_4): re4, b8a
  - B09 (water_vapor): wv, b09
  - B11 (swir1): short_wave_infrared_1, b11
  - B12 (swir2): short_wave_infrared_2, b12

s1:
  - VV (vv): co_pol
  - VH (vh): cross_pol